## Make sure run this notebook in electra repo folder after git clone,

```bash
git clone https://github.com/google-research/electra.git
cd electra
jupyter notebook
```

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [3]:
# !wget https://storage.googleapis.com/electra-data/electra_base.zip
# !unzip electra_base.zip
# !ls electra_base/

In [4]:
import configure_finetuning
from util import training_utils

hparams = {'model_size': 'base', 'vocab_size': 30522}
config = configure_finetuning.FinetuningConfig('electra-base', './electra_base/', **hparams)
bert_config = training_utils.get_bert_config(config)

bert_config.__dict__

{'vocab_size': 30522,
 'hidden_size': 768,
 'num_hidden_layers': 12,
 'num_attention_heads': 12,
 'hidden_act': 'gelu',
 'intermediate_size': 3072,
 'hidden_dropout_prob': 0.1,
 'attention_probs_dropout_prob': 0.1,
 'max_position_embeddings': 512,
 'type_vocab_size': 2,
 'initializer_range': 0.02}

In [5]:
import tensorflow as tf
from model import modeling
from model import optimization

In [6]:
MAX_SEQ_LENGTH = 120

In [7]:
from model import tokenization

tokenizer = tokenization.FullTokenizer(
        vocab_file='electra_base/vocab.txt',
        do_lower_case=True)

In [8]:
import json

with open('../text.json') as fopen:
    data = json.load(fopen)
    
data.keys()

dict_keys(['train_X', 'train_Y', 'test_X', 'test_Y'])

In [9]:
from tqdm import tqdm

def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
              tokens_a.pop()
        else:
              tokens_b.pop()

def get_data(left, right):
    input_ids, input_masks, segment_ids = [], [], []
    for i in tqdm(range(len(left))):
        tokens_a = tokenizer.tokenize(left[i])
        tokens_b = tokenizer.tokenize(right[i])
        _truncate_seq_pair(tokens_a, tokens_b, MAX_SEQ_LENGTH - 3)
        tokens = []
        segment_id = []
        tokens.append("[CLS]")
        segment_id.append(0)
        for token in tokens_a:
            tokens.append(token)
            segment_id.append(0)
        tokens.append("[SEP]")
        segment_id.append(0)
        for token in tokens_b:
            tokens.append(token)
            segment_id.append(1)
        tokens.append("[SEP]")
        segment_id.append(1)
        input_id = tokenizer.convert_tokens_to_ids(tokens)
        input_mask = [1] * len(input_id)

        while len(input_id) < MAX_SEQ_LENGTH:
            input_id.append(0)
            input_mask.append(0)
            segment_id.append(0)

        input_ids.append(input_id)
        input_masks.append(input_mask)
        segment_ids.append(segment_id)
    return input_ids, input_masks, segment_ids

In [10]:
left, right = [], []
for i in range(len(data['train_X'])):
    l, r = data['train_X'][i].split(' <> ')
    left.append(l)
    right.append(r)
    
train_ids, train_masks, segment_train = get_data(left, right)

100%|██████████| 261802/261802 [02:42<00:00, 1608.50it/s]


In [11]:
left, right = [], []
for i in range(len(data['test_X'])):
    l, r = data['test_X'][i].split(' <> ')
    left.append(l)
    right.append(r)
    
test_ids, test_masks, segment_test = get_data(left, right)

100%|██████████| 13395/13395 [00:08<00:00, 1669.02it/s]


In [12]:
from finetune import task_builder

In [13]:
tasks = task_builder.get_tasks(config)
tasks[0].name

'chunk'

In [18]:
batch_size = 60
epoch = 10
num_train_steps = int(len(train_ids) / batch_size * epoch)

class Model:
    def __init__(
        self,
        dimension_output,
    ):
        self.X = tf.placeholder(tf.int32, [None, None])
        self.segment_ids = tf.placeholder(tf.int32, [None, None])
        self.input_masks = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None, None])
        self.batch_size = tf.shape(self.X)[0]
        
        model = modeling.BertModel(
            bert_config=bert_config,
            is_training=False,
            input_ids=self.X,
            input_mask=self.input_masks,
            token_type_ids=self.segment_ids,
            use_one_hot_embeddings=False)
        
        output_layer = model.get_pooled_output()
        
        with tf.variable_scope("task_specific/classify"):
            self.out = tf.layers.dense(output_layer, bert_config.hidden_size)
            self.out = tf.nn.l2_normalize(self.out, 1)
            self.logits = tf.layers.dense(self.out,dimension_output,use_bias=False,
                                          kernel_constraint=tf.keras.constraints.unit_norm())
            
        self.gamma = 64
        self.margin = 0.25
        self.O_p = 1 + self.margin
        self.O_n = -self.margin
        self.Delta_p = 1 - self.margin
        self.Delta_n = self.margin
        
        self.batch_idxs = tf.expand_dims(
          tf.range(0, self.batch_size, dtype=tf.int32), 1)  # shape [batch,1]
        idxs = tf.concat([self.batch_idxs, tf.cast(self.Y, tf.int32)], 1)
        sp = tf.expand_dims(tf.gather_nd(self.logits, idxs), 1)
        mask = tf.logical_not(
            tf.scatter_nd(idxs, tf.ones(tf.shape(idxs)[0], tf.bool),
                          tf.shape(self.logits)))

        sn = tf.reshape(tf.boolean_mask(self.logits, mask), (self.batch_size, -1))

        alpha_p = tf.nn.relu(self.O_p - tf.stop_gradient(sp))
        alpha_n = tf.nn.relu(tf.stop_gradient(sn) - self.O_n)

        r_sp_m = alpha_p * (sp - self.Delta_p)
        r_sn_m = alpha_n * (sn - self.Delta_n)
        _Z = tf.concat([r_sn_m, r_sp_m], 1)
        _Z = _Z * self.gamma
        # sum all similarity
        logZ = tf.math.reduce_logsumexp(_Z, 1, keepdims=True)
        # remove sn_p from all sum similarity
        self.cost = -r_sp_m * self.gamma + logZ
        self.cost = tf.reduce_mean(self.cost[:,0])
        
        self.optimizer = optimization.create_optimizer(
              self.cost, config.learning_rate, num_train_steps,
              weight_decay_rate=config.weight_decay_rate,
              use_tpu=config.use_tpu,
              warmup_proportion=config.warmup_proportion,
              layerwise_lr_decay_power=config.layerwise_lr_decay,
              n_transformer_layers=bert_config.num_hidden_layers
          )
        
        correct_pred = tf.equal(
            tf.argmax(self.logits, 1, output_type = tf.int32), self.Y[:,0]
        )
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [19]:
dimension_output = 2

tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(
    dimension_output,
)

sess.run(tf.global_variables_initializer())

/home/husein/.local/lib/python3.6/site-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [20]:
var_lists = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = 'electra')
saver = tf.train.Saver(var_list = var_lists)
saver.restore(sess, 'electra_base/electra_base')

INFO:tensorflow:Restoring parameters from electra_base/electra_base


In [21]:
labels = ['contradiction', 'entailment']

train_Y = data['train_Y']
test_Y = data['test_Y']

train_Y = [labels.index(i) for i in train_Y]
test_Y = [labels.index(i) for i in test_Y]

In [22]:
from tqdm import tqdm
import time
import numpy as np

tqdm._instances.clear()
EARLY_STOPPING, CURRENT_CHECKPOINT, CURRENT_ACC, EPOCH = 1, 0, 0, 0

while True:
    lasttime = time.time()
    if CURRENT_CHECKPOINT == EARLY_STOPPING:
        print('break epoch:%d\n' % (EPOCH))
        break

    train_acc, train_loss, test_acc, test_loss = [], [], [], []
    pbar = tqdm(
        range(0, len(train_ids), batch_size), desc = 'train minibatch loop'
    )
    for i in pbar:
        index = min(i + batch_size, len(train_ids))
        batch_x = train_ids[i: index]
        batch_masks = train_masks[i: index]
        batch_segment = segment_train[i: index]
        batch_y = train_Y[i: index]
        batch_y = np.expand_dims(batch_y,1)
        acc, cost, _ = sess.run(
            [model.accuracy, model.cost, model.optimizer],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
                model.segment_ids: batch_segment,
                model.input_masks: batch_masks
            },
        )
        assert not np.isnan(cost)
        train_loss.append(cost)
        train_acc.append(acc)
        pbar.set_postfix(cost = cost, accuracy = acc)
    
    pbar = tqdm(range(0, len(test_ids), batch_size), desc = 'test minibatch loop')
    for i in pbar:
        index = min(i + batch_size, len(test_ids))
        batch_x = test_ids[i: index]
        batch_masks = test_masks[i: index]
        batch_segment = segment_test[i: index]
        batch_y = test_Y[i: index]
        batch_y = np.expand_dims(batch_y,1)
        acc, cost = sess.run(
            [model.accuracy, model.cost],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
                model.segment_ids: batch_segment,
                model.input_masks: batch_masks
            },
        )
        test_loss.append(cost)
        test_acc.append(acc)
        pbar.set_postfix(cost = cost, accuracy = acc)

    train_loss = np.mean(train_loss)
    train_acc = np.mean(train_acc)
    test_loss = np.mean(test_loss)
    test_acc = np.mean(test_acc)
    
    if test_acc > CURRENT_ACC:
        print(
            'epoch: %d, pass acc: %f, current acc: %f'
            % (EPOCH, CURRENT_ACC, test_acc)
        )
        CURRENT_ACC = test_acc
        CURRENT_CHECKPOINT = 0
    else:
        CURRENT_CHECKPOINT += 1
        
    print('time taken:', time.time() - lasttime)
    print(
        'epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'
        % (EPOCH, train_loss, train_acc, test_loss, test_acc)
    )
    EPOCH += 1

train minibatch loop:   0%|          | 0/4364 [00:00<?, ?it/s]

epoch: 0, pass acc: 0.000000, current acc: 0.956027
time taken: 2126.4241914749146
epoch: 0, training loss: 4.702116, training acc: 0.933515, valid loss: 2.930228, valid acc: 0.956027



test minibatch loop: 100%|██████████| 224/224 [00:34<00:00,  6.49it/s, accuracy=1, cost=0.119]    


time taken: 2165.476673603058
epoch: 1, training loss: 2.282007, training acc: 0.967206, valid loss: 3.151913, valid acc: 0.952306

break epoch:2

